In [9]:
import numpy as np
import pandas as pd

matrix = np.array([[2.7, 3.3, 3.4, 3.0, 3.1, 3.4, 3.0, 2.7],
                    [3.9, 2.8, 2.6, 4.0, 2.6, 3.0, 3.2, 3.0],
                    [2.5, 2.9, 3.4, 3.0, 3.2, 3.1, 3.2, 2.8]])
df = pd.DataFrame(matrix, columns=[f'Var{i+1}' for i in range(matrix.shape[1])])
print(df)

   Var1  Var2  Var3  Var4  Var5  Var6  Var7  Var8
0   2.7   3.3   3.4   3.0   3.1   3.4   3.0   2.7
1   3.9   2.8   2.6   4.0   2.6   3.0   3.2   3.0
2   2.5   2.9   3.4   3.0   3.2   3.1   3.2   2.8


### DISTANCES FOR BINARY VARIABLES

In [10]:
media = np.mean(df, axis = 0)
df_bin = df
for i, col in enumerate(df.columns):
    df_bin[col] = np.where(df[col] > media[col], 1, 0)

print(df_bin)

   Var1  Var2  Var3  Var4  Var5  Var6  Var7  Var8
0     0     1     1     0     1     1     0     0
1     1     0     0     1     0     0     1     1
2     0     0     1     0     1     0     1     0


In [42]:
def similarity(matrix_bin, method): 
    # method: 'jaccard', 'tanimoto', 'simple_matching', 'russel_rao', 'dice'
    matrix_bin = df.values
    nrow, ncol = matrix_bin.shape
    similarities = np.zeros((nrow, nrow))
    
    # Mapeamento de parâmetros com base no método escolhido
    params = {
        'jaccard': (0, 0, 1),
        'tanimoto': (1, 1, 2),
        'simple_matching': (1, 1, 1),
        'russel_rao': (0, 1, 1),
        'dice': (0, 0, 0.5)
    }
    
    if method not in params:
        raise ValueError(f"Método desconhecido: {method}")

    delta1, delta2, gamma = params[method]

    for i in range(nrow):
        similarities[i][i] = 1  # Similaridade de si mesmo é 1
        for j in range(i + 1, nrow):
            # Computa as variáveis a1, a2, a3, a4
            a1 = np.sum((matrix_bin[i] == 1) & (matrix_bin[j] == 1))
            a2 = np.sum((matrix_bin[i] == 0) & (matrix_bin[j] == 1))
            a3 = np.sum((matrix_bin[i] == 1) & (matrix_bin[j] == 0))
            a4 = np.sum((matrix_bin[i] == 0) & (matrix_bin[j] == 0))

            # Calcula a similaridade com base no método
            similarity = (a1 + (delta1 * a4)) / (a1 + (delta2 * a4) + (gamma * (a2 + a3)))
            
            # Atribui valores simétricos
            #print(i,j, similarity)
            similarities[i][j] = similarity
            similarities[j][i] = similarity
    
    return similarities

# Exemplo de chamada
similarity(df_bin, 'jaccard')


array([[1.        , 0.        , 0.4       ],
       [0.        , 1.        , 0.16666667],
       [0.4       , 0.16666667, 1.        ]])

In [22]:
similarity(df_bin, 'tanimoto')

array([[1.        , 0.        , 0.45454545],
       [0.        , 1.        , 0.23076923],
       [0.45454545, 0.23076923, 1.        ]])

In [23]:
similarity(df_bin, 'simple_matching')

array([[1.   , 0.   , 0.625],
       [0.   , 1.   , 0.375],
       [0.625, 0.375, 1.   ]])

### Posso utilizar a função de jaccard já pronta

In [28]:
from scipy.spatial.distance import pdist, squareform

jaccard_distances = pdist(df, metric='matching')
# Converta a matriz de distância para uma forma quadrada
jaccard_distances_square = 1-squareform(jaccard_distances)# - 1: to similarity
print(jaccard_distances_square)

[[1.    0.    0.625]
 [0.    1.    0.375]
 [0.625 0.375 1.   ]]


### DISTANCES FOR CONTINUOUS VARIABLES

In [36]:
def distances(matriz):
    matriz= df.values
    n = matriz.shape[0]  # Número de vetores
    distancias = np.zeros((n, n))  # Matriz para armazenar as distâncias
    for i in range(n):
        for j in range(i + 1, n):
            distancia = np.linalg.norm(matriz[i] - matriz[j])
            distancias[i, j] = distancia
            distancias[j, i] = distancia  # A distância é simétrica
    return distancias

# Calculando as distâncias
distancias = distances(df)

print("Matriz de distâncias entre todos os vetores:")
print(distancias)

Matriz de distâncias entre todos os vetores:
[[0.         2.82842712 1.73205081]
 [2.82842712 0.         2.23606798]
 [1.73205081 2.23606798 0.        ]]


In [40]:
distances = pdist(df, 'euclidean')
# Converta a matriz de distância para uma forma quadrada
distances_square = squareform(distances)
print(distances_square)

[[0.         2.82842712 1.73205081]
 [2.82842712 0.         2.23606798]
 [1.73205081 2.23606798 0.        ]]


# FINAL CONCLUSIONS

Você pode calcular todas essas distancias pela função pdist(), tanto para variáveis contínuas como binárias